In [169]:
import sys
sys.path.append('../_amigocloud')

In [170]:
from amigocloud import AmigoCloud
import pandas as pd
from shapely import wkb

import geopandas as gpd

from datetime import datetime, timedelta
from shapely import wkb
from shapely.geometry import Polygon, MultiPolygon

In [171]:
# ruta de carpera donde estan las llaves
ruta = open('../ruta.txt')
ruta = ruta.readlines()[0]

In [172]:
# leer token
acceso = open(ruta + '/_keys/api_amigocloud.key','r')
api_token = acceso.readlines()[0]
acceso.close()
amigocloud = AmigoCloud(token=api_token)
amigocloud

In [173]:
def convertir_wkb(wkb_data):
    geom = wkb.loads(wkb_data, hex=True)
    # Si la geometría es un MultiPolygon con un solo polígono, conviértelo a Polígono
    if isinstance(geom, MultiPolygon) and len(geom.geoms) == 1:
        geom = geom.geoms[0]
    return geom

In [272]:
semana = 20

In [273]:
query = f"select cast(split_part(insp.semana_3ra, 'S', 2) as integer) sem_reg, cast(split_part(prop.propiedad, ' / ', 1) as integer) unidad_01, split_part(prop.propiedad, ' / ', 2) unidad_02, cast(split_part(insp.canhero, ' / ', 1) as integer) unidad_03, split_part(insp.canhero, ' / ',2) unidad_04, lote.lote unidad_05, lote.superficie area, lote.fecha_siembra::date fecha_ini, 'SIEMBRA 2024' origen, lote.geometria geometry from dataset_307560 insp inner join dataset_307562 prop on insp.amigo_id = prop.insp_ref_prop inner join dataset_307563 lote on prop.amigo_id = lote.prop_ref_lote where insp.estado_3ra_insp = 'APROBADO' and (lote.estado_de_siembra = 'CAÑA NACIDA' or lote.estado_de_siembra = 'CAÑA NO NACIDA') and cast(split_part(insp.semana_3ra, 'S', 2) as integer) = {semana}"
query_consulta = {'query': query}
select = amigocloud.get('https://app.amigocloud.com/api/v1/projects/32672/sql', query_consulta)
data = select['data']

In [276]:
df = pd.DataFrame(data)
df['geometry'] = df['geometry'].apply(convertir_wkb)
data = gpd.GeoDataFrame(df, geometry='geometry', crs='EPSG:4326')
gdf_utm = data.to_crs('EPSG:32720')
gdf_utm['fecha_ini'] = pd.to_datetime(gdf_utm['fecha_ini'])
now = datetime.now()
gdf_utm['dias'] = (now - gdf_utm['fecha_ini']).dt.days
gdf_utm['fecha_ini'] = gdf_utm['fecha_ini'].dt.strftime('%Y-%m-%d')

In [216]:
#file_name = r'G:\Ingenio Azucarero Guabira S.A\UTEA - SEMANAL - EQUIPO AVIACION UTEA\Trichogramma\2024\SHP\SIEMBRAS.shp'

# Exporta el GeoDataFrame como un shapefile
#gdf_utm.to_file(file_name)

In [290]:
file_name = r'G:\Ingenio Azucarero Guabira S.A\UTEA - SEMANAL - EQUIPO AVIACION UTEA\Trichogramma\2024\SHP\SIEMBRAS.shp'
gdf_siembras = gpd.read_file(file_name)
gdf_siembras = pd.concat([gdf_siembras, gdf_utm], ignore_index=True)
gdf_siembras.to_file(file_name)